In [1]:
# Import libraries
import requests
import urllib.request
import time
from bs4 import BeautifulSoup

import pandas as pd
from dateutil.parser import parse

countries = [
    'portugal','spain','italy','france','germany','uk',
    'us','iran','china','south-korea',
]

superes = {}

for country in countries:
    
    print (country)

    # Set the URL you want to webscrape from
    url = 'https://www.worldometers.info/coronavirus/country/%s'%country

    # Connect to the URL
    response = requests.get(url)

    # Parse HTML and save to BeautifulSoup object¶
    soup = BeautifulSoup(response.text)

    res = pd.DataFrame()

    cases, deaths = [], []
    for ul_tag in soup.find_all('ul', {'class':'news_ul'})[:3]:
        i=0
        for li_tag in ul_tag.find_all('li', {'class':'news_li'}):
            i+=1
            if i==3: break
            strong_tag = li_tag.find_all('strong')
            cases += [strong_tag[0].text.split(' new case')[0]]
            deaths += [strong_tag[1].text.split(' new death')[0]]
    res['cases'] = [np.nan if c==country else int(c) for c in cases]
    res['deaths'] = [np.nan if d==country else int(d) for d in deaths]
    
    dates = []
    for div_tag in soup.find_all('div'):
        for d in div_tag:
            if d.name == 'div' and d.get('class', '') == ['news_date']:
                dates += [d.text.split(' (')[0]]
    res['date'] = [str(d)[:10] 
                   for d in pd.to_datetime([parse(x) for x in dates[:len(cases)]], 
                                           format='%Y%m%d')]
    print (res.set_index('date'))
    print ()
    
    superes[country] = res.set_index('date')

portugal
            cases  deaths
date                     
2020-04-05    754      29
2020-04-04    638      20
2020-04-03    852      37

spain
            cases  deaths
date                     
2020-04-05   4591     471
2020-04-04   6969     749
2020-04-03   7134     850

italy
            cases  deaths
date                     
2020-04-04   4805     681
2020-04-03   4585     766
2020-04-02   4668     760

france
            cases  deaths
date                     
2020-04-04   7788    1053
2020-04-03  23060    1120
2020-04-02   2116    1355

germany
            cases  deaths
date                     
2020-04-05    982      34
2020-04-04   4933     169
2020-04-03   6365     168

uk
            cases  deaths
date                     
2020-04-05   5903     619
2020-04-04   3735     708
2020-04-03   4450     684

us
            cases  deaths
date                     
2020-04-05    850      20
2020-04-04  34196    1331
2020-04-03  32284    1045

iran
            cases  deaths
date      

In [2]:
from datetime import timedelta, date
import numpy as np

xls = pd.ExcelFile("coronav_.xls")
cases = xls.parse('cases')[['date']+countries]
deaths = xls.parse('deaths')[['date']+countries]
recovery = xls.parse('recovery')[['date']+countries]
cases['date'] = cases['date'].astype(str)
deaths['date'] = deaths['date'].astype(str)
recovery['date'] = recovery['date'].astype(str)

today = date.today()
lastdate = cases.date.iloc[-1]
iterations = (pd.to_datetime(today)-pd.to_datetime(lastdate)).days

writer = pd.ExcelWriter("coronav_.xls", engine='xlsxwriter')
    
for tmp_sheet, name in zip([cases,deaths],['cases','deaths']):
    
    for i in range(1,iterations):
        print (i,name)
        tmp_date = pd.to_datetime(lastdate) + timedelta(days=i)
        tmp_country = [str(tmp_date)[:10]]
        print (tmp_country[0], end=' ')
          
        for country in countries:
            print (country, end=' ')
            
            if country not in ['china','south-korea']: 
                
                try:
                    tmp_country += [superes[country].loc[tmp_country[0],name]]
                except KeyError:
                    tmp_country += [np.nan]
                    
            else:
                tmp_country += [np.nan]
                
        print ('\n',tmp_country)
    
        tmp_sheet = tmp_sheet.append(
            pd.DataFrame([tmp_country],columns=['date']+countries)
        )
        
        tmp_sheet.to_excel(writer, name)

recovery.to_excel(writer, 'recovery')
writer.save()

1 cases
2020-04-04 portugal spain italy france germany uk us iran china south-korea 
 ['2020-04-04', 638, 6969, 4805, 7788, 4933, 3735, 34196, 2560, nan, nan]
1 deaths
2020-04-04 portugal spain italy france germany uk us iran china south-korea 
 ['2020-04-04', 20, 749, 681, 1053, 169, 708, 1331, 158, nan, nan]


In [3]:
tmp_sheet

,date,portugal,spain,italy,france,germany,uk,us,iran,china,south-korea
0,2020-01-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-01-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN
2,2020-01-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,NaN
3,2020-01-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0,NaN
4,2020-01-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
69,2020-03-31,20.0,589.0,837.0,499.0,130.0,381.0,912.0,141.0,NaN,NaN
70,2020-04-01,27.0,923.0,727.0,509.0,156.0,563.0,1049.0,138.0,NaN,NaN
71,2020-04-02,22.0,961.0,760.0,1355.0,176.0,569.0,968.0,124.0,NaN,NaN
72,2020-04-03,37.0,850.0,766.0,1120.0,168.0,684.0,1321.0,134.0,NaN,NaN
